In [21]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:


import json

# Load the dataset from the uploaded file
file_path = "/content/drive/MyDrive/cleaned_sinhala_words.json"  # Make sure this path is correct
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.readlines()

import re
import json

# Load Sinhala word dictionary from JSON file
def load_dictionary(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# Calculate Levenshtein Distance
def levenshtein_distance(word1, word2):
    n, m = len(word1), len(word2)
    dp = [[0] * (m + 1) for _ in range(n + 1)]

    for i in range(n + 1):
        dp[i][0] = i
    for j in range(m + 1):
        dp[0][j] = j

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[n][m]

# Spell Checker using Levenshtein Distance
def spell_check(word, dictionary):
    min_distance = float("inf")
    closest_match = word

    for dict_word in dictionary:
        distance = levenshtein_distance(word, dict_word)
        if distance < min_distance:
            min_distance = distance
            closest_match = dict_word

    # Consider a match only if the distance is small enough (e.g., 2 or less)
    if min_distance <= 2:
        return closest_match
    return word  # Return original word if no close match is found

# Correct a sentence's spelling
def correct_spelling(sentence, dictionary):
    words = sentence.split()
    corrected_words = [spell_check(word, dictionary) for word in words]
    return " ".join(corrected_words)

# Define Grammar Rules
def apply_grammar_rules(sentence):
    # Rule 1: If subject is "මම", sentence should end with "මී"
    if "මම" in sentence:
        sentence = re.sub(r'ය$', '', sentence)  # Remove incorrect ending
        if not sentence.endswith("මී"):
            sentence += "මී"

    # Rule 2: If subject is "අපි", sentence should end with "මු"
    elif "අපි" in sentence:
        sentence = re.sub(r'ය$', '', sentence)  # Remove incorrect ending
        if not sentence.endswith("මු"):
            sentence += "මු"

    return sentence

# Paragraph Correction
def correct_paragraph(paragraph, dictionary):
    sentences = paragraph.split(".")
    incorrect_words = []
    corrected_words = []

    corrected_sentences = []
    for sentence in sentences:
        original_sentence = sentence.strip()
        if not original_sentence:
            continue

        # Step 1: Correct spelling
        sentence_with_correct_spelling = correct_spelling(original_sentence, dictionary)

        # Step 2: Apply grammar rules
        corrected_sentence = apply_grammar_rules(sentence_with_correct_spelling)

        # Identify changed words for both spelling and grammar
        if original_sentence != corrected_sentence:
            original_words = original_sentence.split()
            corrected_final_words = corrected_sentence.split()

            for original, corrected in zip(original_words, corrected_final_words):
                if original != corrected:
                    incorrect_words.append(original)
                    corrected_words.append(corrected)

        corrected_sentences.append(corrected_sentence)

    corrected_paragraph = ". ".join(corrected_sentences) + "."
    return incorrect_words, corrected_words, corrected_paragraph

# Load the dictionary
dictionary_path = "/content/drive/MyDrive/cleaned_sinhala_words.json"
sinhala_dictionary = load_dictionary(dictionary_path)

# Provided Incorrect Paragraphs
paragraphs = [
    "මම සෑම පෝය දියකම සිල් සමාදන් වීමට පන්සල් යන්නෙය. උදෑසනම නිවසින් පිටත්වන මා නැවත පැමින්නේ සවස් යාමයේය.",
    "අපි පසුගිය දිනවල අත්න්දු ක්‍රීඩා තරග සදහා පුහුනුම් කටයුතු සිදුකලේය. ඉදිරි සති දෙක තුල තරග පැවැත්වීට නියමිතව තිබේ. සියලුදෙනාගේම බලාපොරොත්තුව තරග සදහා හොදින් මුහුනදී තරගය ජග්‍රහණය කිරීමය.",
    "මම පාසල් නිවාඩු කාලයේදී දෙමව්පියන් සමග අනුරාධපුර වන්දනා චාරිකාවක ගියෙය. ඒ සදහා අප පවුලේ සියලුම සාමාජිකයන් සහබාගී වීම විශේෂත්වකි. හිදී අපි බොහෝ පූජනීය ස්තාන රැසක් වන්නා කලේය.",
    "මේ දින වල සිග්‍රයන් පැතිර යන වසන්ගත උන රෝගයක් නිසා අප විශ්විද්‍යායේ බොහෝ සිසුන් පිරිසක් පීඩාවට පත්වී සිටී. අපි එය අවම කරගැනීමට  හොද සෞඛ්‍යය පුරුදු අනුගමනයකරන්නෙය.",
    "මම උදෑසන ආහාරය සදහා බොහෝවිට බත් ආහාරයට ගැනීමට පුරුදුවී සිටින්නෙය. නමුත් ඉදහිට පිටි වලින් සාදාගත් ආහාරද රිබෝජනයට ගන්නවා. උදේ ආහාරය ලෙස බත් ආහාරයට ගත් විට දවසේ ඉදිරි වැඩ කටයුතු උද්‍යෝගිත්ව සිදුකිරීමට හැකියාව ලැබේ."
]

# Process and Display Results
for i, paragraph in enumerate(paragraphs, start=1):
    incorrect_words, corrected_words, corrected_paragraph = correct_paragraph(paragraph, sinhala_dictionary)

    print(f"Paragraph {i} - Original:\n{paragraph}\n")
    print(f"Incorrect Words: {incorrect_words}")
    print(f"Correct Words: {corrected_words}")
    print(f"Corrected Paragraph:\n{corrected_paragraph}\n{'-'*80}")

Paragraph 1 - Original:
මම සෑම පෝය දියකම සිල් සමාදන් වීමට පන්සල් යන්නෙය. උදෑසනම නිවසින් පිටත්වන මා නැවත පැමින්නේ සවස් යාමයේය.

Incorrect Words: ['දියකම', 'යන්නෙය', 'පැමින්නේ']
Correct Words: ['දිනයකම', 'යන්නෙමී', 'පැමිනෙන්නේ']
Corrected Paragraph:
මම සෑම පෝය දිනයකම සිල් සමාදන් වීමට පන්සල් යන්නෙමී. උදෑසනම නිවසින් පිටත්වන මා නැවත පැමිනෙන්නේ සවස් යාමයේය.
--------------------------------------------------------------------------------
Paragraph 2 - Original:
අපි පසුගිය දිනවල අත්න්දු ක්‍රීඩා තරග සදහා පුහුනුම් කටයුතු සිදුකලේය. ඉදිරි සති දෙක තුල තරග පැවැත්වීට නියමිතව තිබේ. සියලුදෙනාගේම බලාපොරොත්තුව තරග සදහා හොදින් මුහුනදී තරගය ජග්‍රහණය කිරීමය.

Incorrect Words: ['අත්න්දු', 'පුහුනුම්', 'සිදුකලේය', 'පැවැත්වීට', 'ජග්\u200dරහණය']
Correct Words: ['අත්පන්දු', 'පුහුනුවීම්', 'සිදුකලෙමු', 'පැවැත්වීමට', 'ජයග්\u200dරහණය']
Corrected Paragraph:
අපි පසුගිය දිනවල අත්පන්දු ක්‍රීඩා තරග සදහා පුහුනුවීම් කටයුතු සිදුකලෙමු. ඉදිරි සති දෙක තුල තරග පැවැත්වීමට නියමිතව තිබේ. සියලුදෙනාගේම බලාපොරොත්තුව තරග සදහා හොදින් මු